---
title: Haversine distance at different altitudes
subtitle: Numerical methods to compute the distance between two points on the sphere, with altitude
date: 2025-03-10
categories: [tutorial, geospatial, mathematics]
image: images/cover.png
toc: true
draft: true
colab: <a href="https://colab.research.google.com/github/SebastianoF/GeoDsBlog/blob/master/posts/gds-2025-03-10-haversine-with-altitude/index.ipynb" target="_blank"><img src="images/colab.svg"></a>
github: <a href="https://github.com/SebastianoF/GeoDsBlog/blob/master/posts/gds-2025-03-10-haversine-with-altitude/index.ipynb" target="_blank">  <img src="images/github.svg"> </a>
twitter-card:
  image: images/cover.png
---


![](images/cover.png)

# Introduction


In [13]:
from typing_extensions import Annotated

from pydantic import AfterValidator, BaseModel, ValidationError

import numpy as np


def between_minus_180_and_180(lon: float) -> float:
    if lon < -180 or lon > 180:
        raise ValueError(f"Longitude {lon} must be between -180 and +180")
    return lon


def between_minus_pi_and_pi(theta: float) -> float:
    if theta < -np.pi or theta > np.pi:
        raise ValueError(f"Longitude {theta} must be between -pi and +pi")
    return theta


def between_minus_90_and_90(lat: float) -> float:
    if lat < -90 or lat > 90:
        raise ValueError(f"Latitude {lat} must be between -90 and +90")
    return lat


def between_minus_half_pi_and_half_pi(phi: float) -> float:
    if phi < -np.pi / 2 or phi > np.pi / 2:
        raise ValueError(f"Latitude {phi} must be between -pi/1 and +pi/2")
    return phi


def positive(alt: float) -> float:
    if alt < 0:
        raise ValueError(f"Altitude {alt} must be positive")
    return alt


class PointDeg(BaseModel):
    lon: Annotated[float, AfterValidator(between_minus_180_and_180)]
    lat: Annotated[float, AfterValidator(between_minus_90_and_90)]
    alt: Annotated[float, AfterValidator(positive)]

    class Config:
        frozen = True


class PointRad(BaseModel):
    theta: Annotated[float, AfterValidator(between_minus_pi_and_pi)]
    phi: Annotated[float, AfterValidator(between_minus_half_pi_and_half_pi)]
    alt: Annotated[float, AfterValidator(positive)]

    class Config:
        frozen = True


def deg2rad(point: PointDeg) -> PointRad:
    return PointRad(
        theta=float(np.deg2rad(point.lon)),
        phi=float(np.deg2rad(point.lat)),
        alt=point.alt,
    )


def rad2deg(point: PointRad) -> PointDeg:
    return PointDeg(
        lon=float(np.deg2rad(point.theta)),
        lat=float(np.deg2rad(point.phi)),
        alt=point.alt,
    )


# do some testing
# try:
#     PointDeg(lonlat=-)
# except ValidationError as err:
#     print(err)
#     """
#     1 validation error for Model
#     number
#       Value error, 1 is not an even number [type=value_error, input_value=1, input_type=int]
#    """


In [14]:
a = PointDeg(lat=12, lon=90, alt=4)

In [15]:
np.rad2deg(np.deg2rad(369))


np.float64(369.0)

In [16]:
def hav_function(theta_A: float, phi_A: float, theta_B: float, phi_B: float) -> float:
    hav_rad = lambda x: np.sin(x / 2) ** 2
    return hav_rad(phi_B - phi_A) + (1 - hav_rad(phi_B - phi_A) - hav_rad(phi_B + phi_A)) * hav_rad(theta_B - theta_A)


def haversine_distance(p1: PointDeg, p2: PointDeg, R_Km) -> float:
    theta_1, phi_1, theta_2, phi_2 = map(np.radians, [p1.lon, p1.lat, p2.lon, p2.lat])
    return 2 * R_Km * np.arcsin(np.sqrt(hav_function(theta_1, phi_1, theta_2, phi_2)))

In [17]:
# given two points
p_A = PointDeg(lon=0.0, lat=0.0, alt=0)
p_B = PointDeg(lon=5.0, lat=5.0, alt=5)

In [25]:
R_Km = 6371
R_min = np.min([p_A.alt + R_Km, p_B.alt + R_Km])
delta_H = np.abs(p_A.alt - p_B.alt)
M = 3
print(f"radius lowest point:    {R_min} (Km)")
print(f"difference in altitude: {delta_H} (Km)")

radius lowest point:    6371.0 (Km)
difference in altitude: 5.0 (Km)


In [ ]:
D = haversine_distance(p_A, p_B, R_Km)
d = D / (2**M)

print(D, d)


785.7672208422621 98.22090260528276


In [ ]:
def beta(i):
    return i * d * delta_H / D


def alpha(i):
    return i * d * delta_H / D  # in progress


# doing the heavylifting means that there is not much code left at the end.